## d) Vamos a elaborar dos diccionarios diferentes:


El primero de ellos tendrá una estructura clave-valor donde la clave es un número romano del código numérico correspondiente y el valor es una de las causas de muerte agrupadas, donde ahora se incluirán las qua antes quedaban descartadas como: 
"082  XVI.Afecciones originadas en el periodo perinatal"

El segundo tendrá una clave que sea una cuaterna: El número de la causa de la muerte según la descripción anterior, el sexo, el rango de edad y el año, como un entero. El valor asociado será el número de fallecimientos de esas causas, sexo, rango de edad y año.

El dataframe utilizado es como el utilizado en el apartado anterior pero incluyendo causas extra de muerte agrupadas como la citada anteriormente y con las columnas con información de edad y sexo presentes.


------------------------------------------------------------------------------------------------------------------------

Vamos a comenzar modificando la función definida previamente "es_grupo_y_no_total".

Ahora tiene que incluir otras filas y por lo tanto vamos a utilizar otra condición para el filtrado, que en este caso va a ser:

   - Que aparezca un punto en algún momento del nombre de la causa de muerte.
   
   - Que no aparezca el string "Todas las causas" en el nombre de la causa de muerte.


Hemos quitado la restricción de que aparezcan formaciones: tres dígitos guión tres dígitos. Por lo demás, la función es exactamente igual.

In [1]:
import re

def es_grupo_y_no_total(causa):
    
    """
    Función que comprueba si un determinado string cumple las siguientes condiciones:
    
       - El nombre contiene un caracter "." en algún momento.
       - No totaliza **todas** las causas de fallecimiento.
    
    Que quedan sintetizadas en las varaibles "patron" y "patron2" definidas durante la función.
        
    Parameters
    ----------
    causa : string
        String con la causa de muerte, a analizar para ver si cumple las condiciones.
    
    Returns
    -------
    Un valor boolean indicando si cumple las condiciones (True) o no (False).
    
    
    Example
    -------
    >>> es_grupo_y_no_total("001-008 I.Enfermedades infecciosas y parasitarias")
    True
    >>> es_grupo_y_no_total("001-102 I-XXII.Todas las causas Mujeres De 10 a 14 años 1983 308")
    False
    >>> es_grupo_y_no_total("XV.Embarazo")
    True
    """
    
    # Definimos nuestras expresiones a reconocer, la primera es una expresión regular que quiere decir:
    # Contiene un caracter punto ".", como es un caracter reservado hay que denotarlo con "\.".

    patron = "\."
    patron2 = "Todas las causas"

    # Lanzamos un bucle que busca, en primer lugar, si el string contiene un punto ".".
    # En caso de que lo cumpla, comprueba si el string contiene el string prohibido: "Todas las causas".
    # Hay filas que contienen puntos suspensivos, son las que hacen referencia al VIH, para que no se cuelen
    # imponemos una condición expresamente para ello en nuestro filtro.
    
    if re.search(patron, causa)!=None:
        
        if re.search(patron2, causa)!=None:
            return(False)
    
        else:
            if re.search("VIH", causa)!=None:
                return(False)
            else:
                return(True)

    else:
        return(False)

Vamos a elborar nuestro diccionario a partir de un DF de partida que es igual que el del apartado anterior pero con las ligeras modificaciones ya citadas, por lo tanto necesitamos utilizar de nuevo dos funciones que ya hemos definido previamente.

In [2]:
def romano_a_entero(num):
    
    """
    Función que transforma string interpretados como números romanos a números enteros desde el 1 hasta el 1000.
        
    Parameters
    ----------
    num : string
        String que representa un número romano.
    
    Returns
    -------
    valor: int
        Equivalente entero al número romano.
    string
        Si el número introducido no es un numeral romano nos lo indicará.
    
    
    Example
    -------
    >>> romano_a_entero("IV")
    4
    >>> romano_a_entero("CXXIX")
    129
    """

    # Definimos un diccionario con los dígitos romanos. Observar que como tienen una lógica aditiva y sustractiva, 
    # es más útil definir los que provienen de lógica sustractiva por separado en el rango 1(I)-1000(M)
    
    romanos = {'IV':4,'IX':9,'XL':40,'XC':90,'CD':400,'CM':900, 'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000}
    
    # Inicializamos algunas variables:
    
    i = 0
    valor = 0
    
    # Lanzamos un bucle de lectura, obsérvese el proceso, lee bloques de 2 en dos y determina si ese bloque está en 
    # nuestro diccionario de dígitos, en caso de que lo esté, lo interpreta con el valor indicado por el diccionario
    # y lo añade al valor acumulado previo y continúa con la lectura del string en el valor posterior a ese dígito.
    # Si no lo está, coge únicamente el primero de ellos, lo interpreta, lo añade y continúa desde el siguiente a ese.
    
    # En caso de que un valor de entrada no sea un dígito romano, nos devuelve un mensaje que nos lo indica.
    
    while i < len(num):
        if num[i:i+2] in romanos:
            valor = valor + romanos[num[i:i+2]]
            i = i+2
            
        else:
            if num[i] in romanos:
                valor = valor + romanos[num[i]]
                i = i+1
            else:
                return("El valor a interpretar no es un número romano")
            
    return(valor)

In [4]:
def romano(string):
    
    """
    Función que toma un string con números y letras en mayúsculas y extrae de él la primera
    secuencia de caracteres consecutivos en mayúsculas que encuentra.
        
    Parameters
    ----------
    string : string
        String que representa un código numérico con números árabes seguidos de numerales romanos.
    
    Returns
    -------
    palabra: string
        Primera coincidencia de string que compongan un numeral romano en la secuencia de entrada.
    None
        En caso de no encontrar un numeral romano (Secuencia de letras en mayúsculas).
    
    
    Example
    -------
    >>> romano("678-123 XXV")
    XXV
    >>> romano("1234 CXXIX")
    CXXIX
    >>> romano("1234 X-XV")
    X
    >>> romano("1234")
    None
    """
    
    # Utilizamos la función search, el primer argumento es una expresión regular que indica que busque secuencias de
    # letras de la A a la Z en mayúsculas, el segundo indica el string en el que buscar. En caso de no encontrar resultados
    # devuelve None, si los encuentra devuelve la secuencia encontrada. (Esta función genera una salida que nos da los
    # índices de la secuencia encontrada y la secuencia encontrada. La manera que he encontrado de guardar la secuencia
    # es con la función group, en este caso con el argumento 0 para que sea el primer grupo que es el único que
    # encuentra ya que search solo guarda la primera ocurrencia.
    # Posiblemente existan métodos más elegantes de hacerlo, pero este me funciona y no parece generar problemas
    # de rendimiento.)
    
    if re.search("[A-Z]+", string) == None:
        return(None)
    else:
        palabra = re.search("[A-Z]+", string).group(0)
        return(palabra)

El funcionamiento de estas funciones ha sido aclarado detalladamente en apartados previos ("romano_a_entero": 2a), "romano":3)) y por lo tanto NO van a ser detalladas en este apartado, por favor, para obtener documentación acerca de ellas consultar los apartados indicados.

In [5]:
import re
import pandas as pd

def cargar_datos_mod(nombre_csv):
    
    """
    Función que toma como argumento el nombre del csv a importar y devuelve un PandasDF con las columnas:
       ("Código", "Num.romano", "Num.árabe", "Nombre", "Periodo", "Sexo", "Edad", "Total").
       
       - "Código"(str): Código numérico de la causa de muerte con numerales índices enteros más romanos.
       - "Num.romano"(str): Parte correspondiente al numeral romano del "Código".
       - "Num.árabe"(int): Parte correspondiente al numeral árabe del "Código".
       - "Nombre"(str): Nombre de la causa de muerte, sin numerales.
       - "Periodo"(int): Año que referencia el periodo temporal de la magnitud "Total".
       - "Sexo"(str): Sexo de los fallecidos en esa categoría.
       - "Edad"(str): Intervalo de edades de los fallecidos
       - "Total"(int): Total de fallecimientos en el periodo dado por la causa correspondiente.
       
    Adicionalmente devuelve un parámetro denominado "eliminadas" que se corresponde con el número de filas del DF
    original que no pasaron el filtro de la función "es_grupo_y_no_total" y por lo tanto no figuran en el DF final.
        
    Parameters
    ----------
    nombre_csv : string
        String que representa el nombre del csv a importar.
    
    Returns
    -------
    (DF_limpio, eliminadas): (string, int)
        DF_limpio: Un PandasDF que contiene las columnas detalladas anteriormente.
        eliminadas: Número de filas que figuraban en el DF original menos las del DF final.
    
    """
    
    # Comenzamos cargando el csv y definiendo los tipos adecuadamente como ya se vio en el apartado a).
    
    
    DF_sucio = pd.read_csv(nombre_csv, delimiter = ";",dtype = {"Total": str})
    
    DF_sucio["Total"] = DF_sucio["Total"].replace(regex=["\."],value="")
    
    DF_sucio = DF_sucio.astype({"Causa de muerte":"string",
                                "Sexo":"string",
                                "Edad":"string",
                                "Periodo":"int32",
                                "Total": "int64"})
    
    # Elaboramos una lista con las causas de muerte a partir de la correspondiente columna para pasarla de argumento
    # a la función que elaborará la máscara "es_grupo_y_no_total" que fue definida en el apartado b2).
    
    lista = DF_sucio["Causa de muerte"]
    
    cond = [(es_grupo_y_no_total(i)) for i in lista]
    
    # Utilizamos esta máscara para quedarnos solo con las líneas deseadas. El proceso es sencillo, ya que la máscara tiene
    # una logitud similar a la de la columna "Causa de muerte", ya que ha sido elaborada pasando cada elemento de
    # esta columna como argumento a la función "es_grupo_y_no_total" secuencialmente. Ya comentamos que este sería el 
    # proceso a seguir y el objetivo final de la función elaborada en el apartado 2b).
    
    DF_limpio = DF_sucio[cond]
    
    # Creamos dos listas vacías listas para llenarlas con el siguiente bucle, contendrán los elementos que van a figurar
    # en las columnas 
    
    Numeral = []
    Strings = []
    
    # Utilizamos la función split para separar el contenido de la primera columna DF_limpio.iloc[i,0] secuencialmente
    # en dos partes que figuran en una lista, la primera es la parte (lista[0]) numérica del código que figurará en 
    # la columna "Código" y la segunda (lista[1]) es la parte del nombre del código que irá a la columna "Nombre".
    # El proceso se hace añadiendo cada uno de los elementos de estos pares a sus respectivas listas, Numeral y Strings
    # definidas previamente y tras estar estas completamente construidas se asignarán a sus respectivas columnas del DF.
    # El split se hace tomando como caracter de separación ".", y como es un caracter reservado tenemos que denotarlo 
    # como "\."
    
    
    for i in range(len(DF_limpio)):
        lista = re.split("\.", DF_limpio.iloc[i,0])
        
        Numeral.append(lista[0])
        Strings.append(lista[1])
        
    DF_limpio["Código"] = Numeral
    DF_limpio["Nombre"] = Strings
    
    # Ahora aplicamos la función "romano" definida y explicada previamente para separar la parte correspondiente al numeral romano
    # de la columna "Código". Guardamos los resultados en una nueva columna "Num.romano".
        
    DF_limpio["Num.romano"] = [romano(i) for i in DF_limpio["Código"]]
    
    # Y traducimos la columna "Num.romano" a valores enteros y los guardamos en la columna "Num.árabe".
    
    DF_limpio["Num.árabe"] = [romano_a_entero(i) for i in DF_limpio["Num.romano"]]
    
    
    # Hacemos drop también de esta columna antigua que ya hemos descompuesto.
    
    DF_limpio.drop("Causa de muerte", axis=1)
    
    # Reordenamos las columnas que siguen presentes para que aparezcan en el orden deseado.
    
    DF_limpio = DF_limpio.reindex(columns = ["Código", "Num.romano", "Num.árabe", "Nombre", "Periodo","Sexo","Edad", "Total"])
    
    # E imponemos los tipos para asegurarnos de que el DF tiene el formato y forma que deseamos.
    
    DF_limpio = DF_limpio.astype({"Código":"string",
                                  "Num.romano":"string",
                                  "Num.árabe":"int32",
                                  "Nombre":"string",
                                  "Periodo":"int32",
                                  "Sexo":"string",
                                  "Edad":"string",
                                  "Total":"int64"})
    
    # Computamos el total de filas eliminadas por el proceso de filtrado con "es_grupo_y_no_total"
        
    eliminadas = (len(DF_sucio)-len(DF_limpio))

    return(DF_limpio, eliminadas)

A continuación ejecutamos la función e importamos nuestros datos.
La razón del siguiente warning está detallada en el apartado 3).

In [6]:
import warnings
warnings.filterwarnings('ignore')

datos, num_lin_descartadas = cargar_datos_mod("ine_mortalidad_espanna.csv")

Las filas incluidas y las que quedaron fuera:

In [8]:
print("Número de líneas eliminadas:", num_lin_descartadas)
print("Longitud del DF final:", len(datos))

Número de líneas eliminadas: 257400
Longitud del DF final: 43758


Y tenemos nuestros datos correctamente enfrascados en un PandasDF, con las columnas apropiadas.

In [7]:
datos

,Código,Num.romano,Num.árabe,Nombre,Periodo,Sexo,Edad,Total
2574,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2018,Total,Todas las edades,6398
2575,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2017,Total,Todas las edades,6819
2576,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2016,Total,Todas las edades,7033
2577,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2015,Total,Todas las edades,7567
2578,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2014,Total,Todas las edades,6508
...,...,...,...,...,...,...,...,...
267691,090-102 XX,XX,20,Causas externas de mortalidad,1984,Mujeres,95 y más años,52
267692,090-102 XX,XX,20,Causas externas de mortalidad,1983,Mujeres,95 y más años,60
267693,090-102 XX,XX,20,Causas externas de mortalidad,1982,Mujeres,95 y más años,61
267694,090-102 XX,XX,20,Causas externas de mortalidad,1981,Mujeres,95 y más años,63


Consultamos tipos que nunca está de más:

In [9]:
datos.dtypes

Código        string
Num.romano    string
Num.árabe      int32
Nombre        string
Periodo        int32
Sexo          string
Edad          string
Total          int64
dtype: object

Y estamos en plenas condiciones de elaborar nuestros diccionarios, así que vamos a ello:

### Diccionario 1:

Lo voy a plantear de la siguiente manera: 
    
Queremos un diccionario que contenga como claves los numerales arábigos de cada grupo de causas y como valor el nombre de la causa en sí, por lo que vamos a elaborar un DF que enumere los diferentes nombres de causas junto a sus correspondientes arábigos.

In [12]:
aux_DF=datos.drop_duplicates(["Nombre"])[["Num.árabe", "Nombre"]]
aux_DF

,Num.árabe,Nombre
2574,1,Enfermedades infecciosas y parasitarias
25740,2,Tumores
113256,3,Enfermedades de la sangre y de los órganos hem...
120978,4,"Enfermedades endocrinas, nutricionales y metab..."
128700,5,Trastornos mentales y del comportamiento
141570,6,Enfermedades del sistema nervioso y de los órg...
151866,9,Enfermedades del sistema circulatorio
177606,10,Enfermedades del sistema respiratorio
195624,11,Enfermedades del sistema digestivo
211068,12,Enfermedades de la piel y del tejido subcutáneo


Como quedan algunos huecos vacíos, por ejemplo, del 6 salta al 9 o del 18 al 20, vamos a hacer manualmente un par de sustituciones:
    

In [17]:
# Casteamos la columna Num.árabe como strings, ya que este DF es auxiliar y solo nos sirve para elaborar claves, como en
# este caso queremos introducir un guión no queda más remedio que hacerlo como string.

aux_DF = aux_DF.astype({"Num.árabe":"string",
                        "Nombre":"string"})

# Hacemos las sustituciones requeridas en las columnas requeridas, sencillo y claro.
                        
aux_DF["Num.árabe"] = aux_DF["Num.árabe"].replace("6", "6-8")
aux_DF["Num.árabe"] = aux_DF["Num.árabe"].replace("18", "18-19")

In [18]:
aux_DF

,Num.árabe,Nombre
2574,1,Enfermedades infecciosas y parasitarias
25740,2,Tumores
113256,3,Enfermedades de la sangre y de los órganos hem...
120978,4,"Enfermedades endocrinas, nutricionales y metab..."
128700,5,Trastornos mentales y del comportamiento
141570,6-8,Enfermedades del sistema nervioso y de los órg...
151866,9,Enfermedades del sistema circulatorio
177606,10,Enfermedades del sistema respiratorio
195624,11,Enfermedades del sistema digestivo
211068,12,Enfermedades de la piel y del tejido subcutáneo


Y ahora componemos nuestro diccionario:

In [19]:
# Creamos un diccionario vacío que yo voy a llamar de la siguiente manera:

dict_causas = {}

# Y comenzamos a llenarlo con las columnas del DF anterior: aux_DF. Básicamente le indicamos que rellene las claves con los
# valores de la primera columna del DF y les asocie de valores los items en la segunda columna de la misma fila.

for i in range(len(aux_DF)):
    dict_causas[aux_DF.iloc[i,0]]=aux_DF.iloc[i,1]

Podemos consultar nuestro diccionario:

In [20]:
dict_causas

{'1': 'Enfermedades infecciosas y parasitarias',
 '2': 'Tumores',
 '3': 'Enfermedades de la sangre y de los órganos hematopoyéticos, y ciertos trastornos que afectan al mecanismo de la inmunidad',
 '4': 'Enfermedades endocrinas, nutricionales y metabólicas',
 '5': 'Trastornos mentales y del comportamiento',
 '6-8': 'Enfermedades del sistema nervioso y de los órganos de los sentidos',
 '9': 'Enfermedades del sistema circulatorio',
 '10': 'Enfermedades del sistema respiratorio',
 '11': 'Enfermedades del sistema digestivo',
 '12': 'Enfermedades de la piel y del tejido subcutáneo',
 '13': 'Enfermedades del sistema osteomuscular y del tejido conjuntivo',
 '14': 'Enfermedades del sistema genitourinario',
 '15': 'Embarazo, parto y puerperio',
 '16': 'Afecciones originadas en el periodo perinatal',
 '17': 'Malformaciones congénitas, deformidades y anomalías cromosómicas',
 '18-19': 'Síntomas, signos y hallazgos anormales clínicos y de laboratorio, no clasificados en otra parte',
 '20': 'Causas

Lo probamos un poquito aunque es sencillo ver a simple vista que va a funcionar bien.

In [26]:
lista = ["1", "6-8", "15", "20"]

for i in lista:
    print(i+":", dict_causas[i])

1: Enfermedades infecciosas y parasitarias
6-8: Enfermedades del sistema nervioso y de los órganos de los sentidos
15: Embarazo, parto y puerperio
20: Causas externas de mortalidad


Podemos consultar la lista de claves y valores.

In [27]:
key_list = list(dict_causas.keys())
val_list = list(dict_causas.values())

In [30]:
for i in range(len(key_list)):
    print(key_list[i]+":", val_list[i])

1: Enfermedades infecciosas y parasitarias
2: Tumores
3: Enfermedades de la sangre y de los órganos hematopoyéticos, y ciertos trastornos que afectan al mecanismo de la inmunidad
4: Enfermedades endocrinas, nutricionales y metabólicas
5: Trastornos mentales y del comportamiento
6-8: Enfermedades del sistema nervioso y de los órganos de los sentidos
9: Enfermedades del sistema circulatorio
10: Enfermedades del sistema respiratorio
11: Enfermedades del sistema digestivo
12: Enfermedades de la piel y del tejido subcutáneo
13: Enfermedades del sistema osteomuscular y del tejido conjuntivo
14: Enfermedades del sistema genitourinario
15: Embarazo, parto y puerperio
16: Afecciones originadas en el periodo perinatal
17: Malformaciones congénitas, deformidades y anomalías cromosómicas
18-19: Síntomas, signos y hallazgos anormales clínicos y de laboratorio, no clasificados en otra parte
20: Causas externas de mortalidad


### Diccionario 2:

Tendrá una clave que sea una cuaterna: El número de la causa de la muerte según la descripción anterior, el sexo, el rango de edad y el año, como un entero.

El valor asociado será el número de fallecimientos de esas causas, sexo, rango de edad y año.

Vamos a utilizar el DF del apartado c) ligeramente modificado, para cada una de las claves haremos lo siguiente:
   - Al número de causa ########.
   - Al sexo no le haremos ninguna modificación.
   - El rango de edad lo castearemos como un string, tras ello haremos sustituciones para ponerlo en formato (A,B), y para finalizar lo castearemos como tuplas de números enteros.
   - El año lo trataremos sencillamente com oun entero.

In [31]:
abc_DF=datos.drop_duplicates(["Edad"])[["Edad"]]
abc_DF

,Edad
2574,Todas las edades
2613,Menos de 1 año
2652,De 1 a 4 años
2691,De 5 a 9 años
2730,De 10 a 14 años
2769,De 15 a 19 años
2808,De 20 a 24 años
2847,De 25 a 29 años
2886,De 30 a 34 años
2925,De 35 a 39 años


In [49]:
datos_dict["Edad"] = datos_dict["Edad"].replace("De 45 a 49 años", (45,49))

TypeError: Invalid "to_replace" type: 'str'

In [43]:
abb = datos_dict.drop_duplicates(["Edad"])[["Edad"]]
abb

,Edad
2574,Todas las edades
2613,Menos de 1 año
2652,De 1 a 4 años
2691,De 5 a 9 años
2730,De 10 a 14 años
2769,De 15 a 19 años
2808,De 20 a 24 años
2847,De 25 a 29 años
2886,De 30 a 34 años
2925,De 35 a 39 años
